  
  
  
  


# Shiny App: Song Suggester, part II: Building the app in R.

***
## About the Project

Now it is easier to work in R. There, the program will focus mostly on creating a UI and leveraging the dataframes from our previous Python code. The app allows users to enter an artist's name and select a geographical market, then uses Python to fetch related artists and their top tracks. Then, it processes the returned data to display the most popular tracks and a list of related artists in an interactive web interface.


First, we declare the necessary libraries:

In [ ]:
library(shiny)
library(reticulate)
library(dplyr)
library(DT)

Secondly, we create the elements we will find in the UI and create our output commands for the resulting tables.

In [ ]:
source_python("api_calls_Deploy.py")

ui <- fluidPage(
  titlePanel("Spotify Related Artists and Tracks"),
  textInput("artistName", "Enter Artist Name:"),
  selectInput("marketSelect", "Select Market:",
              choices = c("AU", "CA", "US", "GB", "DE", "FR", "JP", "MX"),  # Add more markets as needed
              selected = ""),
  actionButton("searchButton", "Search"),
  DTOutput("topTracksTable"),
  tableOutput("relatedArtistsTable")
)

Finally, on the server function we make sure we are receiving the two dataframes output by out Python code and then we render the output tables.

In [ ]:
server <- function(input, output, session) {
  
  observeEvent(input$searchButton, {
    req(input$artistName)
    market <- input$marketSelect
    
    print(paste("Selected market:", market))
    
    artist_data <- py$main(input$artistName, market)
    
    if (is.list(artist_data) && length(artist_data) >= 1) {
      # Ensure the first element is a data frame
      if (is.data.frame(artist_data[[1]])) {
        top_tracks_df <- artist_data[[1]]
        
        print(str(top_tracks_df))
        top_tracks_df$track_url <- sprintf('<a href="%s" target="_blank">Link</a>', top_tracks_df$track_url)
        
        # Apply dplyr filtering
        processed_tracks_df <- top_tracks_df %>%
          arrange(desc(popularity)) %>%
          group_by(artist_name) %>%
          slice_head(n = 1) %>%
          ungroup()
        
        
        print(str(processed_tracks_df))
        print(head(processed_tracks_df))
        
        # Render the top tracks table
        output$topTracksTable <- renderDT({
          processed_tracks_df
        }, escape = FALSE)
      }
      
      # Check and render the related artists table
      if (length(artist_data) >= 2 && is.data.frame(artist_data[[2]])) {
        related_artists_df <- artist_data[[2]]
        output$relatedArtistsTable <- renderTable({
          related_artists_df
        })
      }
    } else {
      print("Error: Unexpected or no data returned from Python script.")
    }
  })
}

We run the app with the following command:

In [ ]:
shinyApp(ui, server)